# Estudo de caso Amazon Sales - Previsão de vendas diárias usando Prophet

O dataset para este estudo foi obtido no endereço https://www.kaggle.com/datasets/karkavelrajaj/amazon-sales-dataset

Usaremos o algoritimo Prophet para previsão https://facebook.github.io/prophet/docs/quick_start.html

In [ ]:
import locale
locale.setlocale(locale.LC_NUMERIC, 'pt_BR.UTF-8')

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ignora os warnigns
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
# Configura parâmetros do matplot
plt.rcParams['axes.formatter.use_locale'] = True

# Configura o plot com o estilo seaborn ou ggplot
#plt.style.use('seaborn')
#plt.style.use('ggplot')

In [ ]:
path = os.path.join(os.getcwd())
print(path)

## Carregando o dataset

In [ ]:
df = pd.read_csv('Amazon_Sale_Report.csv', sep=',', encoding='utf8', usecols=['Date', 'Amount'], parse_dates=['Date'])
df.shape

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe().round()

In [ ]:
df.isna().sum()

In [ ]:
print('% de dados ausentes')
print(df.isna().sum() / len(df))

## Limpeza e tratamento

O dataset contém cerca de 6% de registros ausentes na coluna [**Amount**]. Levando em consideração que o dataset tem cerca de 128k registros, vamos apenas remover os registros ausentes para simplificar. Logo em seguida, vamos renomear as colunas.

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
# Renomear as colunas [Date] e [Amount]
df.columns = ['Data', 'Valor']

### Resample

O dataset contém várias entradas de vendas do dia a dia. A seguir iremos fazer um redimensionamento dos dados, somando todas as vendas por dia.

In [ ]:
df2 = df.resample(on='Data', rule='D').sum()
df2

In [ ]:
df2.describe().round()

### Visualizando

Vamos plotar um gráfico para visualizar a soma de venda diária

In [ ]:
fig = plt.figure(figsize=(12,5))

plt.ticklabel_format(style='plain')

sns.lineplot(x='Data', y='Valor', data=df2, ax=fig.gca())

plt.title('Vendas por dia')
plt.ylabel('Valor')
plt.xlabel('Data')

plt.tight_layout()
plt.show()

Abaixo, vamos plotar um gráfico do tipo *boxplot* que vai nos ajudar a identificar valores outliers nos dados.

In [ ]:
fig = plt.figure()

plt.ticklabel_format(style='plain')

sns.boxplot(df2, ax=fig.gca())

plt.tight_layout()
plt.show()

Através do *boxplot* acima, identificamos que há apenas 2 valores outliers no dataset. A seguir vamos usar o método de cálculo do intervalo interquartil para definir um limite inferior e superior, e com esses limites definidos, vamos remover os valores fora do range.

In [ ]:
q1 = df2['Valor'].quantile(0.25)
q3 = df2['Valor'].quantile(0.75)
iqr = q3 - q1
limite_inferior = q1 - (iqr * 1.5)
limite_superior = q3 + (iqr * 1.5)

print('Valor Q1:', q1)
print('Valor Q3:', q3)
print('Valor IQR:', iqr)
print('Limite inferior:', limite_inferior)
print('Limite superior:', limite_superior)

outliers = df2.loc[(df2['Valor'] < limite_inferior) | (df2['Valor'] > limite_superior)]

print('Outliers encontrados:', len(outliers))


### Removendo valores outliers

Com os limites definidos e outliers identificados, vamos criar um novo dataframe através do *merge()*, obtendo todos os valores do dataframe principal que não estão presentes no dataframe outliers. Dessa forma, nosso novo dataframe não vai conter os valores outliers.

In [ ]:
df3 = pd.merge(df2, outliers, on='Data', how='left')
df3 = df3[df3['Valor_y'].isnull()].drop('Valor_y', axis=1).rename(columns={'Valor_x': 'Valor'})
df3 = df3.copy()
df3.shape

In [ ]:
fig = plt.figure(figsize=(12,5))

plt.ticklabel_format(style='plain')

sns.lineplot(x='Data', y='Valor', data=df3, ax=fig.gca())
plt.axhline(df3['Valor'].mean(), color='orange', linestyle='--', label='Média')

plt.legend()
plt.title('Vendas sem outliers')
plt.xlabel('Data')
plt.ylabel('Valor')
plt.tight_layout()

plt.show()

## Realizando previsões com o Prophet

A partir de agora, com nosso dataset limpo e tratado, iremos realizar o modelo de regressão utilizando o Prophet para prever vendas futuras através dos dados históricos que temos no dataset.

Certifique-se de instalar a *lib* do Prophet abaixo.

In [ ]:
# Instalando o Prophet
#!pip install prophet

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot

In [ ]:
# Backup do dataframe
df4 = df3.copy().reset_index()

O Prophet exige que o dataframe atenda ao padrão especificado, contendo apenas 2 colunas, a primeia sendo do tipo data e a segunda coluna sendo valor numérico alvo.

Também temos que renomear a coluna que contem a data para **ds** e a coluna de valor para **y**.

In [ ]:
df4.rename(columns={'Data':'ds', 'Valor':'y'}, inplace=True)

# Converte para o tipo date
df4['ds'] = pd.to_datetime(df4['ds'].dt.date)

 O Prophet pode ser instanciado com a configuração padrão ou com parâmetros que podem melhorar o resultado dependendo da qualidade dos dados. Na última etapa deste estudo de caso vamos executar uma validação cruzada que vai nos devolver configurações mais adequadas de acordo com o dataset em análise.
 
 *Atenção: a execução dessa validação cruzada pode levar muito tempo para ser concluida dependendo do volume de dados e do horizonte configurado.*

In [ ]:
# Instancia um modelo padrão do Prophet
#model = Prophet()

# Instancia um modelo com parâmetros de configuração que ajudam a melhorar o resultado.
model = Prophet(changepoint_prior_scale=0.3, seasonality_prior_scale=5.0, changepoint_range=0.5)

# Treina o modelo.
model.fit(df4)

In [ ]:
# Exibe os parâmetros atuais da sazonalidade atual do modelo treinado.
model.seasonalities

O modelo gerado pelo Prophet disponibiliza uma função para criar um novo dataset contendo datas futuras a partir do dataset usado no treino. Vamos utilizar essa função para gerar um período de 5 dias.

In [ ]:
future = model.make_future_dataframe(periods=5, freq='D', include_history=True)
future.tail()

### Realizando previsões futuras

Abaixo vamos usar o novo dataset contendo as datas futuras para realizar a previsão.

In [ ]:
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

#### Plotando a tendência e sazonalidade identificadas pelo modelo

In [ ]:
model.plot_components(forecast);

In [ ]:
fig = plt.figure(figsize=(12,5))

plt.ticklabel_format(style='plain')

model.plot(forecast, ax=fig.gca())
add_changepoints_to_plot(fig.gca(), model, forecast)

plt.title('Pontos de mudança de tendência')
plt.xlabel('Período')
plt.ylabel('Faturamento semanal')

plt.tight_layout()
plt.show()

#### Plotando os resultados da previsão

In [ ]:
fig = plt.figure(figsize=(12,6))

plt.ticklabel_format(style='plain')

model.plot(forecast, ax=fig.gca());

plt.title('Resultado final da previsão de vendas')
plt.legend(['vendas', 'previsão', 'limites'])
plt.xlabel('Período')
plt.ylabel('Vendas')

plt.tight_layout()
plt.show()

#### Plot dinâmico do modelo de previsão

Através do pacote *plot_plotly* disponibilizado pelo Prophet, podemos plotar um gráfico dinâmico.

In [ ]:
plot_plotly(model, forecast, xlabel='Período', ylabel='Valor', figsize=(950,650))

## Executando Cross-Validation

O validação cruzada abaixo irá realizar testes para identificar melhores parâmetros para usarmos no modelo.

*Atenção: a execução dessa validação cruzada pode levar muito tempo para ser concluida dependendo do volume de dados e do horizonte configurado.*

In [ ]:
import time
import itertools

start_time = time.time()

param_grid = {  
    'changepoint_prior_scale': [0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0],
    'seasonality_prior_scale': [1.0, 5.0, 10.0, 15.0, 20.0, 25.0],
    'changepoint_range': [0.5, 0.6, 0.7, 0.8, 0.9],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

#cutoff1s = pd.to_datetime(['2021-01-01', '2022-01-01', '2023-01-01'])

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(df4)  # Fit model with given params
    df_cv = cross_validation(m, horizon='5 d', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])
    
# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

end_time = time.time()

In [ ]:
total_time = end_time - start_time
print(f"Tempo total de execução: {total_time:.2f} segundos")

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

# Fim